<a href="https://colab.research.google.com/github/lilrachel1985/Data-Science-Projects/blob/master/Daskexample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dask

In [42]:
import pandas as pd

%time df = pd.read_csv("train.csv")



CPU times: user 841 ms, sys: 83.5 ms, total: 925 ms
Wall time: 954 ms


In [37]:
import dask.dataframe as dd

In [43]:
#Reading a 1GB file using dask
#The Black Friday dataset used here has 5,50,068 rows. On using Dask, the read time reduced more than ten times as compared to using pandas!
%time ddf= dd.read_csv("train.csv")

CPU times: user 8.02 ms, sys: 0 ns, total: 8.02 ms
Wall time: 8.13 ms


In [44]:
# Install with pip
!pip install dask-ml

In [45]:
from dask.distributed import Client
# start a local Dask client
client = Client()

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:33853
INFO:distributed.scheduler:  dashboard at:            127.0.0.1:8787
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:33973'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:46419'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:39949', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:39949
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:43024
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:35117', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:35117
INFO:distributed.core:Startin

In [49]:
!pip install dask-ml[complete]
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [50]:
import numpy as np
import dask.array as da
from sklearn.datasets import make_classification

In [51]:
X_train, y_train = make_classification(
    n_features=2, n_redundant=0, n_informative=2,
    random_state=1, n_clusters_per_class=1, n_samples=1000)
X_train[:5]

array([[ 1.53682958, -1.39869399],
       [ 1.36917601, -0.63734411],
       [ 0.50231787, -0.45910529],
       [ 1.83319262, -1.29808229],
       [ 1.04235568,  1.12152929]])

In [52]:
# Scale up: increase N, the number of times we replicate the data.
N = 100
X_large = da.concatenate([da.from_array(X_train, chunks=X_train.shape)
                          for _ in range(N)])
y_large = da.concatenate([da.from_array(y_train, chunks=y_train.shape)
                          for _ in range(N)])
X_large

dask.array<concatenate, shape=(100000, 2), dtype=float64, chunksize=(1000, 2), chunktype=numpy.ndarray>

In [53]:
from sklearn.linear_model import LogisticRegressionCV
from dask_ml.wrappers import ParallelPostFit

In [54]:
clf = ParallelPostFit(LogisticRegressionCV(cv=3), scoring="r2")

In [55]:
clf.fit(X_train, y_train)

ParallelPostFit(estimator=LogisticRegressionCV(cv=3), scoring='r2')

In [56]:
y_pred = clf.predict(X_large)
y_pred

dask.array<_predict, shape=(100000,), dtype=int64, chunksize=(1000,), chunktype=numpy.ndarray>

In [59]:
%time clf.score(X_large, y_large)

CPU times: user 874 ms, sys: 57.6 ms, total: 932 ms
Wall time: 1.92 s


0.596